# Semi-Automated Crestline Mapping Routine

### By Scott Petersen and Prof. Lynn Carter
#### Contact: Scott Petersen at scott.karl.petersen@gmail.com

### Introduction
This Python Notebook contains all of the code used to automatically map sand dune crestlines, and was used in the completion of Scott Petersen's honors thesis at the University of Arizona, which was advised by Prof. Lynn Carter (Lunar and Plantary Laboratory). 

The logic in this mapping routine is based on work by \[1\] Telfer et al. (2015), who developed a fully automatic crestline mapper in ArcMap using optical orbital data. The crestline mapper presented here uses LIDAR DEMs from the USGS instead, and lacks an inbuilt vectorization routine due to the lack of vectorization capability by ArcGIS Pro.

This code was developed with the intention to automatically extract the properties of several dune fields to parameterize a dune field's response to being imaged by Synthetic Aperture Radar (SAR). We hoped to use these parameters to estimate some of the characterstics of microdune fields on Venus to support work done in \[2\] Petersen and Carter (2025). See \[3\] Weitz et al. (1994) for more detail on microdunes. In practice, we only applied this mapping routine to the dunes in White Sands National Park. We found that a window size of 15, alpha value of 0.5, and a value of 1.5 resulted in the best-fitting crestline map. Detail on these numbers is explained in section "Define Functions."

### Notes for Use
Actual filepath names have been replaced with placeholders, and you will need to manually replace these with your own files. Each placeholder has the characters `PLACEHOLDER` in it, which will hopefully help you locate each instance where a placeholder needs replacement.

There are some steps that are easier to do in ArcGIS directly rather than through arcpy (generally because attempting to run the arcpy function would crash ArcGIS), and instances are noted in both comments and in markdown cells.

Most code here assumes that your ArcGIS rasters are using a NoData value of -9999. While this can be modified, I would personally recommend making sure that this is still the case.

This code was run in ArcGIS's own Python notebook environment, though in principle it should be possible to run in any notebook environment as long as you install and import the arcpy library and set up the arcpy workspace environment variable.

The vectorization step of this routine used Adobe Illustrator, and will require further fine tuning to be accurate. We were not able to get a great vectorization out of Illustrator, but it was the best of the programs we tried.

### Logistical Items
URLs to references are included in comments, but full references are available in a markdown cell at the end of this notebook.

Please cite this code if you use it as:
<br>Petersen, S. K., Carter, L. M.. (2025). Semi-Automated Crestline Mapping Routine. \[Computer Software\]. https://github.com/ScottyP123/Observations-and-Mapping-of-Sedimentary-Features-near-Select-Tesserae-on-Venus

## Setup
These code cells import all used libraries and defines the workspace variable for arcpy. This variable helps point arcpy functions to the right folder. If you are running this code outside of ArcGIS, run the first cell.

In [ ]:
# Run this if you are running this code outside of ArcGIS.
import arcpy

In [ ]:
# Import all used libraries
import numpy as np
import matplotlib as mp
from matplotlib import pyplot as plt
import math
from scipy import stats
import statsmodels.api as sm
from scipy import signal
import time

# Initialize Workspace 

# Define workspace
arcpy.env.workspace= r"PLACEHOLDER_WORKSPACE_FILEPATH"

# I found this to be useful for debugging. It will yield True if arcpy found your geodatabase.
arcpy.Exists(arcpy.env.workspace)

## Define Functions
While more refactoring is probably possible, these are the only two pieces of code that I turned into functions.
### raster_to_filtered_array
The first takes an input of an arcpy Raster type and returns the raster as a one-dimensional numpy ndarray WITHOUT any NoData pixels. This is mostly used for graphing and Kernel Density Estimation. <br>Inputs:
* inraster_path: This is the path to the raster you want to make into one-dimensional array without NoData values.
* nodata_value: This is whatever value the raster uses as NoData. You can find out what this value is for a raster by right-clicking on a raster in ArcGIS's contents pane, selecting Properties, selecting Source, and then looking under Raster Information. The function skips analysis on NoData pixels. There are also ways to set the raster's NoData value. I used -9999 for NoData.
<br>

### classify_strength_DEM_np_refactor
The second function is the main component of the cretline mapper. It uses a moving window to compute the average and standard deviation of the slope of a DEM. Then, it computes whether or not the pixel is "strong" (that is, likely to be a crestline) based on the same criterion in \[1\] Telfer et al. (2015). We use the magnitude of the slope because crestlines are usually immediately adjacent to a dune's slip face, which is where they should be steepest. Strong pixels get written as a 1, everything else gets written as 0. As in \[1\] Telfer et al. (2015), you could also write other values to a "weak" criterion. This code is currently commented out, and you will need to write an if/elif/else statement to write values to strong, weak, and other.
<br>
The strong criterion is $$P > {\alpha}x+{a}{\sigma}$$<br>
The weak criterion is $$P > x+(a-0.25){\sigma}$$<br>
Both of these are from \[1\] Telfer et al. (2015). P is the current pixel's value, x is the average value of all pixels in the window, and ${\sigma}$ is the standard deviation of all the pixels in the window.
<br>Inputs:
* writeraster: an empty arcpy Raster object that the function will write to. You can make an empty arcpy Raster object with `arcpy.Raster(arcpy.Raster(PLACEHOLDER_FILEPATH_TO_DEM_SLOPE).getRasterInfo())`. It is extremely important that you use the raster of the DEM's slope to generate this empty raster to ensure that the raster the function writes to has the same dimensions as the raster it will be reading from..
* dataraster: an ndarray made out of a raster of your DEM's slope. You can generate this by running `arcpy.RasterToNumPyArray(arcpy.Raster(PLACEHOLDER_FILEPATH_TO_DEM_SLOPE), nodata_to_value=PLACEHOLDER_NODATA_VALUE)`. I recommend setting PLACEHOLDER_NODATA_VALUE to -9999.
* a: This is a nondimensional coefficient used to weight the standard deviation of the pixels in the function's analysis window. This is direclty adapted from \[1\] Telfer et al. (2015). Adjustments to this value were found to be necessary to get an ideal result both by this work and by Telfer et al. (2015).
* alpha: This is a nondimensional coefficient used to weight the average of the pixels in the function's analysis window. This is also a direct adaptation from Telfer et al. (2015), though they note that alpha should be between 0 and 1. I see no a priori reason for this to be the case, but nonetheless we still stuck with it in developing this code.
* window_size: The length, in pixels, of the function's analysis window. This should be an odd number to ensure the window is centered on a pixel and not in between pixels. The funciton will automatically make the window of an odd length if it's input is not already odd. If the window goes over the raster's edge, only values within the raster will be used. 
* nodata_value: Generally assumed to be -9999, but still a required input. This is the value in your DEM slope raster that is set to NoData. `arcpy.RasterToNumPyArray()` sets whatever pixel was specified as a NoData value to math.nan.
<br>

### circle_wrap
This function allows two angles to be added together while constraining the output to \[0, 360\], in degrees. We used it to help find the direction the stoss (upwind) side of the dunes was directed.
<br>Inputs
* deg: Some value in degrees. In this case, it was the mode of the image gradient of a DEM slope raster that corresponded to the stoss side of the dunes.
* addVal: Some other value in degrees that gets added to deg. The sum of deg and addVal is computed and then mapped to \[0, 360\]. In this case, addVal were the bounds within which pixels would be masked out.
<br>

### mask_out_aspect
This function takes in a raster of the gradient direction (aspect) of a DEM and removes all pixels WITHIN two bounds.
<br>Inputs
* inraster: This is an arcpy Raster object of the gradient direction raster.
* writeraster:Tthis is another arcpy Raster object, but is empty. It must have the same dimensions as inraster. Like before, it can be run with `arcpy.Raster(arcpy.Raster(PLACEHOLDER_FILEPATH_TO_DEM_ASPECT).getRasterInfo())`
* lower: The lower bound, in degrees, of pixel values to be removed.
* upper: The upper bound, in degrees, of pixel values to be removed.

In [ ]:
def raster_to_array(inraster_path, nodata_value):
    # Converts inraster to a one-dimensional ndarray
    # Read from raster
    inraster_array = arcpy.RasterToNumPyArray(arcpy.sa.Float(inraster_path), nodata_to_value = nodata_value)

    # Flatten the arrays. Apparently ravel() is faster than flatten() and is sufficient here.
    inraster_ravelled = np.ravel(inraster_array)

    # Filter NoData
    inraster_filter = inraster_ravelled != nodata_value
    filtered_inraster = inraster_ravelled[inraster_filter]
    
    # Return the final array
    return filtered_inraster

def classify_strength_DEM_np_refactor(writeraster, dataraster, a, alpha, window_size, nodata_value):
    # * * * * * * * * * * * * * * * * * *
    # ONLY DATARASTER AS NDARRAY VERSION
    # * * * * * * * * * * * * * * * * * *
    
    # Looks at each pixel of dataraster and and computes strenght, then writes this to writeraster.
    # Strength is based on the value of a moving average using tuning parameters a and alpha.
    # Window size is the size of the square of pixels to be included in the moving average
    # REFACTOR: Trying to increase speed with numpy array slicing.
    
    # Window size should be an odd number (otherwise there is no middle pixel)
    # Ensure that it is:
    if window_size % 2 == 0:
        window_size = int(math.floor(window_size)) + 1
    print('Actual window size: ' + str(window_size))
        
    # Log current time for comparison
    # This is not strictly necessary, but this function can take a while to run.
    start_time = time.time()
    print('\nStart time: ' + str(start_time))
    
    # Initialize thresholds
    strongthresh_win = 0
    weakthresh_win = 0
    
    # Debug counter
    debug_counter = 0
    
    # These were mostly for my own debugging, but I am too scared to delete them.
    # Debugging flag
    first_time = False
    # Initialize a list to store all pixel values in the window
    windowvals = []

    # Iteration through writeraster will be done using arcpy's RasterCellIterator. 
    # Adapted from [4] https://pro.arcgis.com/en/pro-app/latest/help/analysis/spatial-analyst/raster-cell-iterator/a-quick-tour-of-using-the-raster-cell-iterator.htm
    # Seems like explicitly handling NoData cells may be necessary, or at least allows me to directly control for it.
    with arcpy.sa.RasterCellIterator({'rasters':[writeraster]}) as rci:
        # Outermost loop: current cell
        for i,j in rci:
            
            # This will print progress every 5 % of the way through a raster.
            # Increment debug counter
            debug_counter += 1
            # Print current percent completion if it's a multiple of 5 percent.
            if (debug_counter/dataraster.size) % 0.05 < 0.00000015:
                print(debug_counter/dataraster.size)
            
            # Logic starts here.
            # First check if the current cell is NoData. If it is, set the corresponding cellin writeraster to NoData. If not, compute pixel strength.
            if dataraster[i,j] == nodata_value:
                # Writing math.nan to a raster will result in that pixle being interpreted as NoData.
                writeraster[i,j] = math.nan
            else:
                # Numpy slicing just returns nan for means for things completely oustide of bounds.
                # Only case when numpy has a problem is if the slicing syntax a:b has a negative a.
                # Also note that slicing includes index a but not b, so a +1 is needed.
                # So, need to control for all possibilities where this can happen.
                
                # Left hand out of bounds
                if (i - window_size <= 0):
                    data_slice = dataraster[0:i+1+window_size, j-window_size:j+1+window_size]
                    
                # Top out of bounds
                elif (j - window_size <= 0):
                    data_slice = dataraster[i-window_size:i+1+window_size, 0:j+1+window_size]
                    
                # Out of bonds on both top and left
                elif (i - window_size <= 0) and (j - window_size <= 0):
                    data_slice = dataraster[0:i+1+window_size, 0:j+1+window_size]
                
                # In bounds, or out of bounds in a way that doesn't cause the numpy slicing to return nan.
                else:
                    data_slice = dataraster[i-window_size:i+1+window_size, j-window_size:j+1+window_size]
                
                # Filtration flattens an ndarray.
                # Filter NoData
                data_slice_filter = data_slice != nodata_value
                filtered_data_slice = data_slice[data_slice_filter]

                # Compute statistics on items in window
                windowmean = filtered_data_slice.mean()
                windowstd = filtered_data_slice.std()
                
                # This is for debugging and not necessary, but as mentioned, I'm too scared to try to remove it.
                # This just prints the full results of one loop of the function on only the first loop.
                if first_time == True:
                    print('\nArray: ')
                    print(data_slice)
                    print('\nFiltered array: ')
                    print(filtered_data_slice)
                    print('\nMean: ' + str(windowmean) +', Std: ' + str(windowstd) + '\n')
                    first_time = False

                # Compute strong candidate
                strongthresh_win = (alpha * windowmean) + (a * windowstd)
                
                # Compute weak candidate
                # Not used in our work, but maybe you will have some use on it. Identical to the weak criterion in [1] Telfer et al. (2015).
                #weakthresh_win = windowmean + ((a - 0.25) * windowstd)

                # Write a strong pixel if current pixel exceeds threshold, and a 0 pixel otherwise
                if dataraster[i,j] > strongthresh_win:
                    writeraster[i,j] = 1
                else:
                    writeraster[i,j] = 0
                    
    # Now outside of cell iteration, log end time
    end_time = time.time()
    print('\nElapsed time: ' + str(end_time - start_time))

def circle_wrap(deg, addVal):
    # Add addVal to deg as an angle restricted to [0, 360] degrees.
    initial_sum = deg + addVal

    # Check for various cases where sum could be outside [0, 360] and adjust.
    # For 360, we'll make it "greater than or equal to" to avoid a degeneracy between 0 and 360.
    if initial_sum >= 360:
        final_sum = initial_sum - 360
        return final_sum
    elif initial_sum < 0:
        final_sum = 360 + initial_sum
        return final_sum
    else:
        return initial_sum
    
def mask_out_aspect(inraster, writeraster, lower, upper):
    # Creates a mask based on an aspect map, inraster. Boundaries are "lower" and "upper."
    # Values between are kept.
    
    with arcpy.sa.RasterCellIterator({'rasters':[inraster, writeraster]}) as rci:
        # Outermost loop: current cell
        for i,j in rci:
            if (inraster[i,j] == -9999) or (math.isnan(inraster[i,j]) == True):
                writeraster[i,j] = math.nan
            elif (inraster[i,j] >= lower) and (inraster[i,j] <= upper):
                writeraster[i,j] = 1
            else:
                writeraster[i,j] = 0

## Compute Strength Map (Once)
The following is the code needed to go from a slope map of your DEM to a map of "strong" pixels. Compute the slope of your DEM using ArcGIS's Surface Parameters function. 

In [ ]:
# Define the NoData value
nd_val = -9999

# Define path to DEM slope raster
trimslope_path = r"PLACEHOLDER_FILEPATH_TO_DEM_SLOPE"

# Create an numpy ndarray version of the DEM slope raster
trimslope_arr = arcpy.RasterToNumPyArray(arcpy.Raster(trimslope_path), nodata_to_value=nd_val)

# Create an empty raster of the same size to write to
# Using .getRasterInfo() just returns an empty raster of the same dimensions as the one it was called on.
writeable_raster = arcpy.Raster(arcpy.Raster(trimslope_path).getRasterInfo())

# Set a and alpha parameters
a = 1
alpha = 1

# Compute the strength of each pixel in the DEM slope raster
# This particular function call will set a = 1, alpha = 1, and window size to 5.
# Change them as you need.
# Arguments: writeraster, dataraster, a, alpha, window_size, nodata_value
classify_strength_DEM_np_refactor(writeable_raster, trimslope_arr, a, alpha, 5, nd_val)

In [ ]:
# After the above block runs, save the resulting raster
# IMPORTANT: Even though a raster may be added to the current map of the strength raster after running 
# classify_strength_DEM_np_refactor(), IT IS NOT YET SAVED! It will vanish if you close ArcGIS, so you need
# to run the save method of the raster.
# I found it useful to save the a and alpha values in the raster's name, and the placeholder filepath
# below gives an example of how you might do that.
writeable_raster.save(fr"PLACEHOLDER_PATH_TO_STRENGTH_RASTER\PLACEHOLDER_NAME_{a}_{alpha}.tif")

## Compute Strength Map (Multiple Times)
To avoid needing to manually adjust values of a and alpha to determine which combination resulted in the best classification, I used the following code to generate every permutation of a, alpha, and window size. Note that this took about an hour to run for 20 combinations.

In [ ]:
# Define the range of values that you want to permute (permutate? iterate?) over.
# Alpha is between 0 and 1, and the window size has to be odd.
# These values are just examples, you can change them.
a_range = [0, 0.5, 1, 1.5, 2]
alpha_range = [0.5, 1]
window_range = [3, 5, 7]

# Initialize a counter for total rasters to make
total = 0

# Set the path to the DEM slope raster, if not already done
trimslope_path = r"PLACEHOLDER_FILEPATH_TO_DEM_SLOPE"

# Define the NoData value, if not already done
nd_val = -9999

# Initialize array of rasters
raster_array = []

# These nested loops turn raster_array into a 3-dimensional array containing each combination of a, alpha, and window
# size that you want to test. It has the following structure: 
# raster_array[index of value of a][index of value of alpha][index of value of window size]
for a_val1 in range(len(a_range)):
    # Add a blank array for the "a axis"
    raster_array.append([])
    for alpha_val1 in range(len(alpha_range)):
        # Add a blank array for the "alpha axis"
        raster_array[a_val1].append([])
        for window_val1 in range(len(window_range)):
            # Add a blank raster with the same dimensions as the DEM slope raster at the coordinates of the current a, alpha,
            # and window size values.
            raster_array[a_val1][alpha_val1].append(arcpy.Raster(arcpy.Raster(trimslope_path).getRasterInfo()))
            total += 1
            
print(f"Total rasters to generate: {total}")

In [ ]:
# Define arrays to hold names for procedurally generating raster file names
# These should correspond to your a_range, alpha_range, and window_range arrays
# Each entry should be a string.
a_name = ['0', '0_5', '1', '1_5', '2']
alpha_name = ['0_5', '1']
window_name = ['3', '5', '7']

In [ ]:
# Now that everything is initialized, start iterating
# This will count progress
counter = 0

# This flag will be set to true if an error occurs while saving the rasters, indicating something really bad has happened
# The below for loop saves the raster at the end of each loop, so it is possible to pick up where you left off if something
# goes wrong.
abort = False

# Iterate over a values
for a_val in range(len(a_range)):
    if abort == True:
        break
    # Iterate over alpha values
    for alpha_val in range(len(alpha_range)):
        if abort == True:
            break
        # Iterate over window size values
        for window_val in range(len(window_range)):
            # Print that a new raster is being generated with the raster's information
            print("---------STARTING NEW---------")
            print(f"{round(counter/total, 2)}::: {a_range[a_val]}, {alpha_range[alpha_val]}, {window_range[window_val]}")
            counter += 1
            
            # Here comes the fun bit
            # Arguments: writeraster, dataraster, a, alpha, window_size, nodata_value
            classify_strength_DEM_np_refactor(raster_array[a_val][alpha_val][window_val], trimslope_arr, a_range[a_val], alpha_range[alpha_val], window_range[window_val], nd_val)
            
            # Save the raster immediately, break out of the loops if something goes wrong.
            # I have had saving fail (usually when I type in a filepath wrong, but sometimes when ArcGIS just decides something else is wrong)
            # so this try/except block is meant to prevent this code from running for several hours without ever actually producing a result.
            try:
                raster_array[a_val][alpha_val][window_val].save(fr"PLACEHOLDER_PATH_TO_STRENGTH_RASTER\NAME_{str(window_name[window_val])}x_{str(a_name[a_val])}a_{str(alpha_name[alpha_val])}alp.tif")
            except:
                print(f"!!! ERROR ENCOUNTERED WHEN SAVING Window: {window_name[window_val]} a: {a_name[a_val]} alpha: {alpha_name[alpha_val]}!!!")
                abort = True
                break
                


# This is just an extra indicator that all of the rasters have been made.
print('************COMPLETE************')

## Mask Stoss Sides of Dunes
At least for the dunes in White Sands, we did not find a permutation of a, alpha, and window size that identified crestlines that also excluded the stoss (upwind) side of the dunes. Therefore, following after \[1\] Telfer et al. (2015), we excluded the stoss side by masking out slopes that were not facing toward the lee side of the dunes. This method may not work for dune fields where there is a significant change in crestline orientation.
<br>We tried using the inbuilt ArcGIS Aspect tool, which is iself a part of the Surface Parameters tool. However, easier-to-process results were achieved using the Sobel filter-based method used by \[1\] Telfer et al. (2015). This method computes the direction of the image gradient at each pixel, then identifies the modal direction corresponding to stoss side of the dunes. Pixels within

In [ ]:
# Define filepath to save gradient direction raster in.
# This raster has not been made yet, but defining its filepath ahead of time helps make the code look better.
# This one is not optional.
gdir_path = r"PLACEHOLDER_GRADIENT_DIRECTION_FILEPATH"


# Defining the below filepaths is optional.
# Define paths to x-axis convolution rasters and y-axis convolution rasters.
# These rasters have not been generated yet! But defining these filepaths ahead of time helps make the code look better.
convX_path = r"PLACEHOLDER_X_CONVOLUTION_RASTER_FILEPATH"
convY_path = r"PLACEHOLDER_Y_CONVOLUTION_RASTER_FILEPATH"

# Define paths to the gradient magnitude raster.
# Again, this has not yet been generated at this point in the code!
# The magnitude of the gradient of the image is not at all necessary for masking out the stoss side of the dunes, but is
# included for reference.
gmag_path = r"PLACEHOLDER_GRADIENT_MAGNITUDE_FILEPATH"

In [ ]:
# Sobel filter an image and compute the gradient direction for each pixel.
# Be sure to cast the final rasters to a float! Will be critical later for gradient magnitude and direction.
# UNRESOLVED: does Convolution not spit out floats? That could be very problematic.
# RESOLVED NOW: It does not, so maybe the input raster needs to be floatified?

# Point to DEM file
active_path = r"C:\Users\Scott Work\Documents\Data_ForArc\Earth\DEMs\USGS_WS_T2.tif"

# First, define x sobel kernel. It should be a list of a list.
# The kernel is from [1] Tefler et al. (2015).
x_sobel = [[1,2,0,-2,-1],[4,8,0,-8,-4],[6,12,0,-12,-6],[4,8,0,-8,-4],[1,2,0,-2,-1]]

# y sobel kernel, as given in Tefler et al. (2015)
y_sobel = [[1,4,6,4,1],[2,8,12,8,2],[0,0,0,0,0],[-2,-8,-12,-8,-2],[-1,-4,-6,-4,-1]]

# Apply x sobel to greyscale image
# It is necessary to cast the raster to a float several times, otherwise the arcpy.sa.Convultion() function causes
# ArcGIS to crash.
convolve_x = arcpy.sa.Float(arcpy.sa.Convolution(arcpy.sa.Float(active_path), kernel=x_sobel))

# Save the x convolution (optional)
#convolve_x.save(convX_path)

# Apply y sobel to greyscale image
convolve_y = arcpy.sa.Float(arcpy.sa.Convolution(arcpy.sa.Float(active_path), kernel=y_sobel))

# Save the y convolution (optional)
#convolve_y.save(convY_path)

# Compute gradient magnitude (optional)
#gradient_mag = arcpy.sa.Float(arcpy.sa.SquareRoot((arcpy.sa.Float(convolve_x)**2)+(arcpy.sa.Float(convolve_y)**2)))
# Save
#gradient_mag.save(gmag_path)

# Compute the gradient direction in degrees. This computation is from \[1\] Telfer et al. (2015)
# arcpy natively operates in radians so conversion to degrees is necessary.
gradient_dir = arcpy.sa.Float(((arcpy.sa.Float(arcpy.sa.ATan2(convolve_y, convolve_x)) * 57.2958) % 360) + 180)

# Save
gradient_dir.save(gdir_path)

# Set NoData value
# IMPORTANT: Without this, the NoData value is some weird default value that can render the raster impossible to interpet!
# Define a ValueTable to hold the nodata values
# This is the only way to add the NoData attribute to a raster. I was not even able to do it manually, despite following
# the ArcGIS documentation exactly.
nd = arcpy.ValueTable(["GPLong", "GPLong"])
nd.addRow([1, nd_val])
print(nd)

# Push the above ValueTable object to the gradient direction raster, setting the nodata value.
arcpy.management.SetRasterProperties(gdir_path, nodata=nd)


In [ ]:
# Now, we will use statsmodels's Kernel Density Estimator to identy modal peaks in the gradient direction raster.
# Conceptually, I think what kernal density estimation does is it makes something like a histogram without bins.

# Convert the gradient direction raster to a one-dimensional numpy array
ravelled_gdirdem = np.ravel(arcpy.RasterToNumPyArray(arcpy.sa.Float(gdir_path), nodata_to_value = -9999))

# Filter out all values less than -1, which should eliminate all NoData pixels from the array.
ravelled_filter = ravelled_gdirdem > -1
arrayified_gdirDEM1 = ravelled_gdirdem[ravelled_filter]

# Filter out any weird values that are above 360 for any reason.
ravelled_filter2 = arrayified_gdirDEM1 < 400
arrayified_gdirDEM = arrayified_gdirDEM1[ravelled_filter2]

# Print to make sure it looks OK.
print(arrayified_gdirDEM)

In [ ]:
# Kernel Density Estimation
# Adapted from the statmodels documentation [5].
kde_sobel = sm.nonparametric.KDEUnivariate(arrayified_gdirDEM)
kde_sobel.fit()

In [ ]:
# Find all resulting maxima using signal processing package from SciPy
kde_maxima = signal.find_peaks(kde_sobel.density)

print("\n*****MAXIMA VALUES*****")
for maximum in kde_maxima[0]:
    print(kde_sobel.support[maximum])

In [ ]:
# This is optional, but it helped us determinie which maxima was from the stoss sides.
# Plot both the KDE and gradient direction raster histogram using matplotlib.
fig, ax1 = plt.subplots(1, 1, figsize=(10,5))#, sharey=True, tight_layout=True)

n_bins = 250

ax1.hist(arrayified_gdirDEM, bins=n_bins)

# Create a second y-axis: https://matplotlib.org/stable/gallery/subplots_axes_and_figures/two_scales.html
ax2 = ax1.twinx()
ax2.plot(kde_sobel.support, kde_sobel.density, color='orange', lw=2)

# Plot vertical lines at each maximum
for maximum in kde_maxima[0]:
    ax2.axvline(kde_sobel.support[maximum], color='black', linestyle='dashed')

plt.show()

At this point, I still was not sure which mode corresponded to the stoss side and which corresponded to the lee side. Even though there were many peaks from the KDE function, it was clear that it was mostly just a form of artifacting, and the tallest peaks corresponded to either the stoss or lee sides of the dunes. I decided to solve this problem with trial and error, since I did not know for certain that 0 degrees on the gradient direction raster actually corresponded to north. There are some indications that it may correspond to east on the arcpy documentation.
<br>To this end, we made masks that would remove pixels within plus or minus 90 degrees of the "low" and "high" KDE peaks, then checked which one worked. "low" and "high" here refer to the degree heading of the peak. For example, the dunes at White Sands had two main peaks at 314 degrees and 135 degrees. The 135 degree peak was the "low" peak, and the 314 degree peak was the "high" peak.
<br>Going plus/minus 90 degrees around the modal peak of the stoss side was from \[1\] Telfer et al. (2015).
<br>Trying to mask out the values using the arcpy.as.Con() function universally resulted in a crash of ArcGIS, so was not used here.

In [ ]:
# Find the index of the low peak being investigated from kde_maxima and set it here.
low_index = int('PLACEHOLDER')

# Do the same for the h.
low_index = int('PLACEHOLDER')

# Set low maxima value
low = kde_sobel.support[kde_maxima[0][2]]

# Set high maxima value
high = kde_sobel.support[kde_maxima[0][4]]

# Create bounds
# Low peak
low_lower = circle_wrap(low, -90)
low_upper = circle_wrap(low, 90)

# High peak
high_lower = circle_wrap(high, -90)
high_upper = circle_wrap(high, 90)

# Check that these are reasonable and correct.
print("Low peak bounds: " + str(low_lower) + ", " + str(low_upper))
print("High peak bounds: " + str(high_lower) + ", " + str(high_upper))

In [ ]:
# Now, create an empty raster to hold the raster crestline map
crestline_map_write = arcpy.Raster(arcpy.Raster(gdir_path).getRasterInfo())

# Create a temporary copy of the gradient direction raster.
# I have had problems where somehow the gradient direction raster's values get modified, so 
# using a temporary copy prevents this from being an issue.
gdir_data = arcpy.Raster(gdir_path)

# Remove all pixels between the selected boundary set to create the raster crestline map.
# If you removed the lee side slopes, then switch boundary sets (i.e. low_lower and low_upper to 
# high_lower and high_upper)
mask_out_aspect(gdir_data, masked_aspect_DEM_write, low_lower, low_upper)

# Save the raster crestline map
masked_aspect_DEM_write.save(r"PLACEHOLDER_CRETLINE_MAP_FILEPATH")

## Vectorization
To continue with analysis, you will need to vectorize the raster crestline map. We used Adobe Illustor's Image Trace tool. Its settings were: mode to “Black and White,” noise to 2 pixels, path to 100%, corners to 90%, and threshold to 254. Then, we ran the "Add Anchor Points" tool to all created paths five times in a row. 
<br> To bring the vectorized map back into ArcGIS, scale it down using a coordinate system of 1 px = 1 degree. Then place it approxmately at the lattitude and longitude coordinates in Illustrator by editing the x/y location (measured in pixels). This will not precisely place the map in the correct location, but it will help get it close. Then, export this file as an AutoCAD .dwg file and add it to your ArcGIS project. Set the resulting CAD Layer's spatial reference to your map's spatial reference (such as WGS84). Use ArcGIS's CAD to Feature Layer tool to convert it from a CAD layer to an actual polyline feature layer. Then, you can move and scale the layer until it overlaps with the dune field.
<br>Further analysis may require breaking the polyline layer into individual polylines, but it also may not.
<br>Finding a better vectorization program may be helpful, as Illustrator had some unusual quirks in its vectorization.

## References

\[1\] Telfer, M. W., Fyfe, R. M., Lewin, S. Automated mapping of linear dunefield morphometric parameters from remotely-sensed data. Aeolian Research, 19, Pp. 215-224. Doi: http://dx.doi.org/10.1016/j.aeolia.2015.03.001
<br><br>
\[2\] Petersen, S. K., Carter, L. M. (2025). Preliminary Observations of Sedimentary Processes near Venusian Tesserae. 56th Lunar and Planetary Science Conference, Abstract 1823. https://www.hou.usra.edu/meetings/lpsc2025/pdf/1823.pdf 
<br><br>
\[3\] Weitz, C. M., Plaut, J. J., Greeley, R., Saunders, R. S. (1994). Dunes and Microdunes on Venus: Why Were So Few Found in the Magellan Data? Icarus, 112, Pp. 282-295. Doi: https://doi.org/10.1006/icar.1994.1181
<br><br>
\[4\] Esri. (n.d.) A quick tour of using Raster Cell Iterator. https://pro.arcgis.com/en/pro-app/latest/help/analysis/spatial-analyst/raster-cell-iterator/a-quick-tour-of-using-the-raster-cell-iterator.htm
<br><br>
\[5\] Perktold, J., Seabold, S., Taylor, J. (2025). Kernel Density Estimation (Version 0.15.0). TypeError and AnyPost. https://www.typeerror.org/docs/statsmodels/examples/notebooks/generated/kernel_density